# 2MRS Group Catalogs

In this notebook we will explore the various 2MRS group catalogs and look at the most massive halos. We'll then compile the group catalogs to be used in the data analysis.

In [1]:
import os, sys

#sys.path.append("/zfs/nrodd/NPTF-ID-Catalog/2MASS/")
# sys.path.append("/group/hepheno/smsharma/Fermi-LSS/2MASS/")


import numpy as np
import pandas as pd
import healpy as hp
import matplotlib.pyplot as plt
from astropy.cosmology import Planck13, z_at_value
from scipy.interpolate import UnivariateSpline, interp1d
from scipy.interpolate import InterpolatedUnivariateSpline
from astropy.io import fits
from tqdm import *
from astropy import units as u
from astropy.coordinates import SkyCoord, Distance

from units import *
import tools
from tools import lb2pix
import halo as hl
import Jpdf
import JBpdf
import Dpdf

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Huchra et al catalog (original 2MRS)

Huchra (original 2MASS) catalog. We don't actually do anything with it, just look at the 2MASS IDs etc to confirm the format.

In [2]:
# fits.open("/group/hepheno/smsharma/Fermi-LSS/AdditionalData/catalog/2mrs_1175_done.fits")[1].data['TMID']

## Cosmicflow

Cosmicflow catalog. Has a bunch of info that we use like common cluster names.

In [3]:
from astropy.io import ascii
cf_df = ascii.read("../DataFiles/Misc/ajaa259at2_mrt.txt").to_pandas()
cf_df.columns.values

array(['Nest', 'o_<DM>-gp', '<DM>-gp', 'e_<DM>-gp', '<Dist>-gp', 'Abell',
       'GName', 'Npv', 'PGC1', 'GLON-gp', 'GLAT-gp', 'SGLON-gp',
       'SGLAT-gp', 'logLKs', 'cf', 'Vsigma-proj', 'R2t-proj',
       '<Vhelio>-gp', '<Vgsr>-gp', '<Vls>-gp', '<Vcmb>-g', '<Vcmba>-gp',
       'Vrms-gp', 'Mvir', 'Mlum', 'LDC', 'HDC', '2M++', 'MK2011', 'Icnt'], dtype=object)

## Tully catalog

Do some data munging to match Tully 2MRS and Tully clusters.

In [4]:
# Tully version of 2MRS table
twomrs_df =  pd.read_csv("../DataFiles/Misc/2MRS1175.csv",delimiter="|")
twomrs_df.head()

,pgc,ID_2MASXJ,RAJ,DEJ,Glon,Glat,SGL,SGB,K_c,H_c,...,r_ext,b/a,Flgs,Type,So,Vhel,e_V,C,Bib_z,Name
0,1,23595879+0042066,359.99500,0.70179,96.87975,-59.54168,293.35040,13.42997,11.503,11.882,...,1.473,0.72,0000,98,NN,24478,32,F,20112MRS.FLWO.0000H,23595879+0042066
1,2,00000168+4716282,0.00695,47.27456,113.95532,-14.69903,341.64417,20.73886,9.683,9.969,...,2.001,0.60,0333,3B_T,ZC,5017,9,N,1991RC3.9.C...0000d,UGC_12889
2,5,00000914+3244182,0.03814,32.73844,110.62066,-28.90421,326.17715,19.78053,10.608,10.912,...,1.461,0.80,0000,-4,ZC,10372,19,N,1999PASP..111..438F,IC_5370
3,12,00000865-0622263,0.03601,-6.37399,90.19206,-65.93002,286.42487,11.35101,11.224,11.497,...,1.609,0.26,0000,4,ZC,6531,45,6,20096dF...C...0000J,g0000086-062226
4,14,00000701+0816448,0.02926,8.27913,101.78542,-52.47268,300.91431,15.36726,10.758,11.066,...,1.602,0.60,0666,-5,ZC,11602,0,N,2000ApJS..129..547B,UGC_12890


In [5]:
# North clusters
twomrs_ng_clusters_df = pd.read_csv("../DataFiles/Misc//2MRS1175NG.csv",delimiter="|")

In [6]:
# South clusters
twomrs_sg_clusters_df = pd.read_csv("../DataFiles/Misc//2MRS1175SG.csv",delimiter="|")

In [7]:
# Combine north and south cluster catalogs
twomrs_g_clusters_df = pd.concat([twomrs_ng_clusters_df, twomrs_sg_clusters_df])

In [8]:
# Get unique galaxy groups
twomrs_g_clusters_Nests = pd.Series.unique(twomrs_g_clusters_df['Nest'])
twomrs_g_clusters_PGC1 = []

# Loop over unique galaxy groups and get the brightest central
for Nest in tqdm(twomrs_g_clusters_Nests):
    twomrs_g_clusters_PGC1.append(twomrs_g_clusters_df[twomrs_g_clusters_df['Nest'] == Nest]['PGC1'].values[0])

100%|██████████| 25486/25486 [00:16<00:00, 1511.90it/s]


In [9]:
# Get relevant cluster property arrays and also cross-match with Tully 2MRS to get names of brightets clusters (this is what is most slow)

twomrs_g_clusters_masses = []
twomrs_g_clusters_redshifts = []
twomrs_g_clusters_Glon = []
twomrs_g_clusters_Glat = []
twomrs_g_clusters_name = []
twomrs_g_clusters_rvir = []
twomrs_g_clusters_nmem = []
twomrs_g_clusters_twomasxj = []
twomrs_g_clusters_bibz = []
cf_GName = []

for PGC1 in tqdm(twomrs_g_clusters_PGC1):
    PGC1_loc = twomrs_g_clusters_df['pgc'] == PGC1
    PGC1_loc_cf = cf_df['PGC1'] == PGC1
    if np.sum(PGC1_loc) == 0: continue
    twomrs_g_clusters_nmem.append(np.sum(twomrs_g_clusters_df['PGC1'] == PGC1))

    twomrs_g_clusters_masses.append(twomrs_g_clusters_df[PGC1_loc]['L_Mass12'].values[0]*10**12)
    
    twomrs_g_clusters_redshifts.append(twomrs_g_clusters_df[PGC1_loc]['Vgp'].values[0]/300000.)
    twomrs_g_clusters_Glon.append(twomrs_g_clusters_df[PGC1_loc]['GLong'].values[0])
    twomrs_g_clusters_Glat.append(twomrs_g_clusters_df[PGC1_loc]['GLat'].values[0])
    twomrs_g_clusters_rvir.append(twomrs_g_clusters_df[PGC1_loc]['R2t'].values[0])
    twomrs_g_clusters_name.append(twomrs_df[twomrs_df['pgc'] == PGC1]['Name'].values[0])
    twomrs_g_clusters_twomasxj.append(twomrs_df[twomrs_df['pgc'] == PGC1]['ID_2MASXJ'].values[0])
    twomrs_g_clusters_bibz.append(twomrs_df[twomrs_df['pgc'] == PGC1]['Bib_z'].values[0])
    GName = cf_df[PGC1_loc_cf]['GName'].values
    if len(GName) != 0:
        cf_GName.append(cf_df[PGC1_loc_cf]['GName'].values[0])
    else:
        cf_GName.append(np.nan)

100%|██████████| 25486/25486 [02:48<00:00, 154.21it/s]


Now loop over halos to get J-factors, cvirs etc from `halo.py`.

In [11]:
# hm_bartels = hl.HaloModel(boost_model='bartels', concentration_model='correa_Planck15')
hm_bartels = hl.HaloModel(boost_model='bartels', concentration_model='correa_Planck15', omega_m=Planck15.Om0, omega_lambda=Planck15.Ode0, h = Planck15.h)

Loading bartels
Interpolating concentration for DarkSky
Using model correa_Planck15


In [15]:
mulog10Jfnb_arr = []
sigmalog10Jfnb_arr = []
mulog10Jf_arr = []
sigmalog10Jf_arr = []

mulog10JBfnb_arr = []
sigmalog10JBfnb_arr = []
mulog10JBf_arr = []
sigmalog10JBf_arr = []

mulog10Df_arr = []
sigmalog10Df_arr = []

twomrs_g_clusters_rvir_der = []
twomrs_g_clusters_cvir_der = []

test = []

# Interpolated cvir errors
cvir_err_arys = np.load("/tigress/nrodd/DM-Catalog-Scan/DataFiles/Misc/cvir_err.npy")
cvir_err_inter = InterpolatedUnivariateSpline(cvir_err_arys[0],cvir_err_arys[1])

Ji = Jpdf.Jc()
JBi = JBpdf.Jc()
Di = Dpdf.Dc()


for i in tqdm(range(len(twomrs_g_clusters_masses))):
    hm_bartels.cvir = -999
    hm_bartels.rvir = -999
        
    mvir = twomrs_g_clusters_masses[i]*M_s # This is in keV...

    # Some clusters have a 0 mass, if so set mvir to be tiny
    if twomrs_g_clusters_masses[i] == 0.:
        mvir = 0.01*M_s
    
    z = twomrs_g_clusters_redshifts[i]
    
    if z < 0.03 and np.log10(mvir/M_s) > 10.1: 
        
        cvir = hm_bartels.c_vir(mvir, z)
        rvir = hm_bartels.r_vir(mvir, z)

        mvir /= M_s # Msun

        mulog10M = np.log10(mvir)
        sigmalog10M = mulog10M*0.01

        mulog10c = np.log10(cvir)
        #sigmalog10c = 0.15 # NB: Might want to update this later!    
        sigmalog10c = cvir_err_inter(np.log10(mvir))

        mulog10Jnb, sigmalog10Jnb, mulog10J, sigmalog10J = Ji.log10Jmusig(mulog10M, sigmalog10M, mulog10c, sigmalog10c, z)
        mulog10JBnb, sigmalog10JBnb, mulog10JB, sigmalog10JB = JBi.log10Jmusig(mulog10M, sigmalog10M, mulog10c, sigmalog10c, z)
        mulog10D, sigmalog10D = Di.log10Dmusig(mulog10M, sigmalog10M, z)
    
    else:
        mulog10Jnb, sigmalog10Jnb, mulog10J, sigmalog10J = -999., -999., -999., -999.
        mulog10JBnb, sigmalog10JBnb, mulog10JB, sigmalog10JB = -999., -999., -999., -999.
        mulog10D, sigmalog10D = -999., -999.
        cvir, rvir = -999, -999
        
    mulog10Jfnb_arr.append(mulog10Jnb)
    sigmalog10Jfnb_arr.append(sigmalog10Jnb)
    mulog10Jf_arr.append(mulog10J)
    sigmalog10Jf_arr.append(sigmalog10J)
    
    mulog10JBfnb_arr.append(mulog10JBnb)
    sigmalog10JBfnb_arr.append(sigmalog10JBnb)
    mulog10JBf_arr.append(mulog10JB)
    sigmalog10JBf_arr.append(sigmalog10JB)
    
    mulog10Df_arr.append(mulog10D)
    sigmalog10Df_arr.append(sigmalog10D)
    
    twomrs_g_clusters_rvir_der.append(rvir)
    twomrs_g_clusters_cvir_der.append(cvir)

  0%|          | 3/25485 [00:00<17:30, 24.25it/s]

Loading bartels
Interpolating concentration for DarkSky
Using model correa_Planck15
Loading bartels
Interpolating concentration for DarkSky
Using model correa_Planck15
Loading bartels
Interpolating concentration for DarkSky
Using model correa_Planck15


100%|██████████| 25485/25485 [04:15<00:00, 99.80it/s] 


In [17]:
print np.min(mulog10Jf_arr), np.max(mulog10Jf_arr)

-999.0 18.9175161859


Make dataframe and add in information we just inferred

In [18]:
nhalos = len(mulog10Jf_arr)

sales = zip(twomrs_g_clusters_name,
            mulog10Jf_arr,
            sigmalog10Jf_arr,
            mulog10Jfnb_arr,
            sigmalog10Jfnb_arr,
            mulog10JBf_arr,
            sigmalog10JBf_arr,
            mulog10JBfnb_arr,
            sigmalog10JBfnb_arr,
            mulog10Df_arr,
            sigmalog10Df_arr,
            np.log10(twomrs_g_clusters_masses),
            twomrs_g_clusters_redshifts,
            twomrs_g_clusters_Nests,
            twomrs_g_clusters_nmem,
            twomrs_g_clusters_Glon,
            twomrs_g_clusters_Glat,
            twomrs_g_clusters_cvir_der,
            np.array(twomrs_g_clusters_rvir_der)/(.001*Mpc),
            np.array(twomrs_g_clusters_rvir_der)/(.001*Mpc)/np.array(twomrs_g_clusters_cvir_der),
            twomrs_g_clusters_twomasxj,
            twomrs_g_clusters_bibz,
            cf_GName)
labels = ["Name", "mulog10J_inf","sigmalog10J_inf","mulog10Jnb_inf","sigmalog10Jnb_inf",
          "mulog10JB_inf","sigmalog10JB_inf","mulog10JBnb_inf","sigmalog10JBnb_inf",
          "mulog10D_inf","sigmalog10D_inf",
          "log10Mvir_inf", "z", "Nest", "Ng", "l", "b", "cvir_inf", "rvir_inf", "rs", "ID_2MASXJ", "Bib_z", "GName"]
df = pd.DataFrame.from_records(sales, columns=labels)
df = df[df["z"] < 0.03] # Cut to avoid the issue with crazy massive halos from overincluding satellite contribution
df = df[df["mulog10J_inf"] != -999.] 
dftop = df.sort(["mulog10J_inf"],ascending=False)[:nhalos]

/tigress/smsharma/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: RuntimeWarning: divide by zero encountered in log10
/tigress/smsharma/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:33: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Add 3FGL-related information

In [20]:
# Load in the 3FGL catalog 
source_3fg_df = pd.read_csv('../DataFiles/Catalogs//3fgl.dat', sep='|', comment='#')

In [21]:
# Remove whitespace from column names
source_3fg_df.rename(columns=lambda x: x.strip(), inplace=True)
for col in source_3fg_df.columns.values:
    try:
        source_3fg_df[col] = source_3fg_df[
            col].map(str.strip)
    except TypeError:
        continue

# Convert to numeric data
source_3fg_df = source_3fg_df.convert_objects(
    convert_numeric=True)

/tigress/smsharma/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [22]:
# Change Virgo and Fornax centers

dftop.loc[dftop['GName'] == 'Virgo','l'] = 283.7777978
dftop.loc[dftop['GName'] == 'Virgo','b'] = 74.4911308

dftop.loc[dftop['GName'] == 'Fornax','l'] = 236.72
dftop.loc[dftop['GName'] == 'Fornax','b'] = -53.64

In [23]:
# Astropy-formatted coordinates of cluster and 3FGL
c2 = SkyCoord("galactic", l=dftop['l']*u.deg, b=dftop['b']*u.deg)
c3 = SkyCoord("galactic", l=source_3fg_df['_Lii']*u.deg, b=source_3fg_df['_Bii']*u.deg)

In [24]:
# Select 3FGL sources within some radius

threefgl0p5deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, .5*u.deg)
for i in range(len(dftop)):
    threefgl0p5deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)

threefgl1deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 1*u.deg)
for i in range(len(dftop)):
    threefgl1deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)

threefgl2deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 2*u.deg)
for i in range(len(dftop)):
    threefgl2deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)
    
threefgl5deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 5*u.deg)
for i in range(len(dftop)):
    threefgl5deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)

In [25]:
# 3FGL sources within x degrees
dftop['3FGL 0.5'] = pd.Series(threefgl0p5deg, index=dftop.index)
dftop['3FGL 1'] = pd.Series(threefgl1deg, index=dftop.index)
dftop['3FGL 2'] = pd.Series(threefgl2deg, index=dftop.index)
#dftop['3FGL 5'] = pd.Series(threefgl5deg, index=dftop.index)

In [26]:
# Number of 3FGL sources within 5 degrees
dftop['N3FGL 5'] = pd.Series([len(x) for x in threefgl5deg], index=dftop.index)

In [27]:
# Angular extension

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=hm_bartels.h*100, Om0=hm_bartels.omega_m)
ang_ext = 2*dftop['rs'].values/(cosmo.angular_diameter_distance(dftop['z']).value*1000)*(180/np.pi)
rvir_ang = dftop['rvir_inf'].values/(cosmo.angular_diameter_distance(dftop['z']).value*1000)*(180/np.pi)

dftop['ang_ext'] = pd.Series(ang_ext, index=dftop.index)
dftop['theta_vir'] = pd.Series(rvir_ang, index=dftop.index)
# dftop['rvir_ang'] = pd.Series(rvir_ang, index=dftop.index)

In [28]:
# M200
dftop['log10M200_inf'] = pd.Series(np.log10(np.array([hm_bartels.M200_cvir(m, z, c) for m,z,c in zip(10**dftop['log10Mvir_inf'].values*M_s, dftop['z'], dftop['cvir_inf'])])/M_s), index=dftop.index)

In [29]:
dftop['ra'] = c2.transform_to('icrs').ra.degree  
dftop['dec'] = c2.transform_to('icrs').dec.degree  

In [30]:
# dftop['bsh'] = 10**dftop.mulog10J_inf/10**dftop.mulog10Jnb_inf

In [31]:
# Boost factor (1+bsh)
# dftop['bsh'] = pd.Series([1+hm_bartels.bsh(m, z).flatten()[0] for m,z in zip(10**dftop['log10Mvir_inf']*M_s,dftop['z'])], index=dftop.index)

In [32]:
dftop.columns.values

array(['Name', 'mulog10J_inf', 'sigmalog10J_inf', 'mulog10Jnb_inf',
       'sigmalog10Jnb_inf', 'mulog10JB_inf', 'sigmalog10JB_inf',
       'mulog10JBnb_inf', 'sigmalog10JBnb_inf', 'mulog10D_inf',
       'sigmalog10D_inf', 'log10Mvir_inf', 'z', 'Nest', 'Ng', 'l', 'b',
       'cvir_inf', 'rvir_inf', 'rs', 'ID_2MASXJ', 'Bib_z', 'GName',
       '3FGL 0.5', '3FGL 1', '3FGL 2', 'N3FGL 5', 'ang_ext', 'theta_vir',
       'log10M200_inf', 'ra', 'dec'], dtype=object)

In [34]:
pd.read_csv("../DataFiles/Catalogs/2MRSTully_ALL_DATAPAPER_Planck15_v4.csv")

,Unnamed: 0,Name,mulog10Jnb_inf,siglog10Jnb_inf,mulog10J_inf,siglog10J_inf,log10Mvir_inf,z,Nest,Ng,...,rvir_inf,rs,ID_2MASXJ,Bib_z,GName,3FGL 0.5,3FGL 1,3FGL 2,N3FGL 5,ang_ext
0,4387,MESSIER_049,18.164248,0.337346,18.913682,0.354394,14.723456,0.004958,100002,192,...,2.130007e+03,3.410498e+02,12294679+0800014,2000MNRAS.313..469S,Virgo,[],[],['3FGL J1226.8+0638' '3FGL J1223.3+0818'],8,1.791865e+00
1,5221,NGC_5128,17.864995,0.350136,18.526417,0.363590,13.658011,0.002614,100350,6,...,9.408498e+02,1.136660e+02,13252775-4301073,1978PASP...90..237G,NaN,['3FGL J1325.4-4301'],['3FGL J1324.0-4330e' '3FGL J1325.4-4301'],['3FGL J1333.8-4417' '3FGL J1324.0-4330e' '3FG...,14,1.129411e+00
2,15807,NGC_1316,17.682939,0.345964,18.394158,0.349048,14.025306,0.004531,200015,49,...,1.246570e+03,1.657499e+02,03224178-3712295,1998A&AS..130..267L,Fornax,['3FGL J0322.5-3721'],['3FGL J0322.5-3721'],['3FGL J0322.5-3721'],4,9.525756e-01
3,2103,MESSIER_081,17.882948,0.367027,18.357049,0.370216,11.260071,0.000263,101368,3,...,1.494500e+02,1.082288e+01,09553318+6903549,1991RC3.9.C...0000d,NaN,[],['3FGL J0955.4+6940'],['3FGL J0955.4+6940'],6,1.063957e+00
4,4415,MESSIER_094,17.670560,0.357400,18.276359,0.376261,12.882525,0.001590,120491,3,...,5.189799e+02,5.200627e+01,12505314+4107125,1993A&A...272...63M,NaN,[],[],[],6,8.488050e-01
5,3622,MESSIER_109,17.599168,0.349559,18.265725,0.364692,13.706718,0.003717,100008,56,...,9.763834e+02,1.195206e+02,11573598+5322282,2001A&A...370..765V,UrsaMajor,[],[],[],4,8.363012e-01
6,17658,ESO_137-_G_006,17.419112,0.331870,18.209407,0.335224,15.103804,0.016896,200002,167,...,2.842570e+03,5.071065e+02,16150386-6054261,2008MNRAS.383..445W,Norma,[],[],['3FGL J1630.3-6126' '3FGL J1603.7-6011'],8,7.933490e-01
7,4833,MESSIER_104,17.462935,0.346526,18.168173,0.374774,13.971740,0.005562,100128,15,...,1.196011e+03,1.569356e+02,12395949-1137230,2000MNRAS.313..469S,NaN,['3FGL J1239.1-1158'],['3FGL J1239.1-1158'],['3FGL J1239.1-1158'],5,7.356519e-01
8,15605,NGC_1275,17.375641,0.332121,18.165509,0.335306,15.082785,0.017429,200001,180,...,2.796667e+03,4.962753e+02,03194823+4130420,1992ApJS...83...29S,Perseus,['3FGL J0319.8+4130'],['3FGL J0316.6+4119' '3FGL J0319.8+4130'],['3FGL J0316.6+4119' '3FGL J0319.8+4130'],7,7.531715e-01
9,4768,NGC_4696,17.404204,0.336041,18.161692,0.356260,14.794488,0.012818,100003,113,...,2.244399e+03,3.680181e+02,12484927-4118399,1991RC3.9.C...0000d,Centaurus,[],[],[],7,7.551791e-01


In [33]:
dftop

,Name,mulog10J_inf,sigmalog10J_inf,mulog10Jnb_inf,sigmalog10Jnb_inf,mulog10JB_inf,sigmalog10JB_inf,mulog10JBnb_inf,sigmalog10JBnb_inf,mulog10D_inf,...,GName,3FGL 0.5,3FGL 1,3FGL 2,N3FGL 5,ang_ext,theta_vir,log10M200_inf,ra,dec
4387,MESSIER_049,18.917516,0.355015,18.168082,0.337998,18.665798,0.384031,17.916364,0.368357,20.126449,...,Virgo,[3FGL J1230.9+1224],[3FGL J1230.9+1224],"[3FGL J1223.2+1215, 3FGL J1230.9+1224, 3FGL J1...",7,1.792896,5.643855,14.632619,187.705935,12.391097
5221,NGC_5128,18.531712,0.364132,17.870290,0.350699,18.302303,0.392697,17.640881,0.380275,19.615731,...,NaN,[3FGL J1325.4-4301],"[3FGL J1324.0-4330e, 3FGL J1325.4-4301]","[3FGL J1333.8-4417, 3FGL J1324.0-4330e, 3FGL J...",14,1.132044,4.721495,13.578999,201.367201,-43.018055
15807,NGC_1316,18.398373,0.349636,17.687155,0.346557,18.161455,0.379381,17.450237,0.376545,19.506446,...,Fornax,[],[3FGL J0336.9-3622],[3FGL J0336.9-3622],6,0.953568,3.613975,13.942642,54.615768,-35.452772
2103,MESSIER_081,18.363829,0.370630,17.889728,0.367445,18.171857,0.396334,17.697756,0.393357,19.209782,...,NaN,[],[3FGL J0955.4+6940],[3FGL J0955.4+6940],6,1.068546,7.431067,11.197744,148.888240,69.065403
4415,MESSIER_094,18.282295,0.376741,17.676497,0.357905,18.067144,0.403871,17.461346,0.386360,19.271859,...,NaN,[],[],[],6,0.851502,4.280728,12.810263,192.720770,41.119887
3622,MESSIER_109,18.270429,0.365235,17.603873,0.350126,18.039997,0.393739,17.373440,0.379766,19.359242,...,UrsaMajor,[],[],[],4,0.837665,3.448111,13.627307,179.399725,53.374401
17658,ESO_137-_G_006,18.206956,0.335896,17.416661,0.332548,17.946568,0.365859,17.156274,0.362788,19.448279,...,Norma,[],[],"[3FGL J1630.3-6126, 3FGL J1603.7-6011]",8,0.788027,2.226523,15.009223,243.765304,-60.907172
4833,MESSIER_104,18.171852,0.375318,17.466614,0.347114,17.935977,0.403167,17.230739,0.377053,19.275338,...,NaN,[3FGL J1239.1-1158],[3FGL J1239.1-1158],[3FGL J1239.1-1158],5,0.735965,2.826388,13.889736,189.997707,-11.623042
15605,NGC_1275,18.162789,0.335976,17.372922,0.332797,17.902826,0.365965,17.112958,0.363048,19.400599,...,Perseus,[3FGL J0319.8+4130],"[3FGL J0316.6+4119, 3FGL J0319.8+4130]","[3FGL J0316.6+4119, 3FGL J0319.8+4130]",7,0.747893,2.124341,14.988537,49.951166,41.511588
4768,NGC_4696,18.161413,0.356879,17.403924,0.336697,17.907785,0.385647,17.150296,0.367051,19.376712,...,Centaurus,[],[],[],7,0.751999,2.311531,14.703490,192.205558,-41.310965


In [25]:
dftop.to_csv("Catalogs/2MRSTully_ALL_DATAPAPER_everything_custom_v5.csv")

In [27]:
pd.read_csv("Catalogs/2MRSTully_ALL_DATAPAPER_everything_custom_v5.csv")

,Unnamed: 0,Name,mulog10J_inf,sigmalog10J_inf,mulog10Jnb_inf,sigmalog10Jnb_inf,mulog10JB_inf,sigmalog10JB_inf,mulog10JBnb_inf,sigmalog10JBnb_inf,...,GName,3FGL 0.5,3FGL 1,3FGL 2,N3FGL 5,ang_ext,theta_vir,log10M200_inf,ra,dec
0,4387,MESSIER_049,18.917516,0.355015,18.168082,0.337998,18.665798,0.384031,17.916364,0.368357,...,Virgo,['3FGL J1230.9+1224'],['3FGL J1230.9+1224'],['3FGL J1223.2+1215' '3FGL J1230.9+1224' '3FGL...,7,1.792896,5.643855,14.632619,187.705935,12.391097
1,5221,NGC_5128,18.531712,0.364132,17.870290,0.350699,18.302303,0.392697,17.640881,0.380275,...,NaN,['3FGL J1325.4-4301'],['3FGL J1324.0-4330e' '3FGL J1325.4-4301'],['3FGL J1333.8-4417' '3FGL J1324.0-4330e' '3FG...,14,1.132044,4.721495,13.578999,201.367201,-43.018055
2,15807,NGC_1316,18.398373,0.349636,17.687155,0.346557,18.161455,0.379381,17.450237,0.376545,...,Fornax,[],['3FGL J0336.9-3622'],['3FGL J0336.9-3622'],6,0.953568,3.613975,13.942642,54.615768,-35.452772
3,2103,MESSIER_081,18.363829,0.370630,17.889728,0.367445,18.171857,0.396334,17.697756,0.393357,...,NaN,[],['3FGL J0955.4+6940'],['3FGL J0955.4+6940'],6,1.068546,7.431067,11.197744,148.888240,69.065403
4,4415,MESSIER_094,18.282295,0.376741,17.676497,0.357905,18.067144,0.403871,17.461346,0.386360,...,NaN,[],[],[],6,0.851502,4.280728,12.810263,192.720770,41.119887
5,3622,MESSIER_109,18.270429,0.365235,17.603873,0.350126,18.039997,0.393739,17.373440,0.379766,...,UrsaMajor,[],[],[],4,0.837665,3.448111,13.627307,179.399725,53.374401
6,17658,ESO_137-_G_006,18.206956,0.335896,17.416661,0.332548,17.946568,0.365859,17.156274,0.362788,...,Norma,[],[],['3FGL J1630.3-6126' '3FGL J1603.7-6011'],8,0.788027,2.226523,15.009223,243.765304,-60.907172
7,4833,MESSIER_104,18.171852,0.375318,17.466614,0.347114,17.935977,0.403167,17.230739,0.377053,...,NaN,['3FGL J1239.1-1158'],['3FGL J1239.1-1158'],['3FGL J1239.1-1158'],5,0.735965,2.826388,13.889736,189.997707,-11.623042
8,15605,NGC_1275,18.162789,0.335976,17.372922,0.332797,17.902826,0.365965,17.112958,0.363048,...,Perseus,['3FGL J0319.8+4130'],['3FGL J0316.6+4119' '3FGL J0319.8+4130'],['3FGL J0316.6+4119' '3FGL J0319.8+4130'],7,0.747893,2.124341,14.988537,49.951166,41.511588
9,4768,NGC_4696,18.161413,0.356879,17.403924,0.336697,17.907785,0.385647,17.150296,0.367051,...,Centaurus,[],[],[],7,0.751999,2.311531,14.703490,192.205558,-41.310965


In [29]:
!pwd

/group/hepheno/smsharma/Fermi-LSS/2MASS/notebooks/2MASS


In [ ]:
dftop['lo']